#### Test RS knn with cosine distance on mean centered data, evaluating with RSME

In [41]:
import numpy as np
import pandas as pd
from reco_systems.filter import filter_df
from reco_systems.user_game_matrix import *
from reco_systems.CF_knn import *
import seaborn as sns
from reco_systems.evaluation import *
from scipy.sparse import csr_matrix

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# import DB et set min_reviews
folder = "database_cleaned"
avis_clean  = pd.read_csv(f"{folder}/avis_clean.csv", index_col=0)
jeux_clean  = pd.read_csv(f"{folder}/jeux_clean.csv", index_col=0)
users       = pd.read_csv(f"trictrac_database/users.csv", names=["Username", "User id"])

min_reviews = 5

In [43]:
# filter data with the minimum reviews
filtered_data = filter_df(avis_clean, min_reviews)

# we center the ratings for the centered cos version
filtered_centrd_data, _ = center_score(filtered_data)

11201 10549
3242 3957
3006 3910
3003 3910


In [44]:
# get the needed matrixes

# matrix_ratings : sparse matrix for nonzero values only: row = users, cols = games
# mask_ratings : mask for NaN values only -> with matrix_ratings, can get ratings = 0
# users/games_table_assoc : association index - ids for the matrix_use

matrix_ratings, mask_ratings, users_table_assoc, games_table_assoc = get_matrix_user_game(filtered_centrd_data)
np.unique(matrix_ratings.todense())

array([-8.75116279, -8.62222222, -8.51428571, ...,  5.66666667,
        5.75      ,  5.91162791])

In [45]:
# we calculate the similarity matrix with cos on the centered values
similarity_matrix, _ = calc_similarity_matrix(matrix_ratings, mask_ratings, "cos")

# cosine similarity range [-1,1] cosine distance = 1 - similarity, range [0, 2] 
np.unique(similarity_matrix) 

array([0.        , 0.04441922, 0.09306476, ..., 1.86065469, 1.86281226,
       1.86330066])

In [33]:
# for the evaluation, we test on the X most popular users (by number of reviews) 
users_count = filtered_centrd_data[["User id", "Game id"]].groupby("User id", as_index=True).count().sort_values(by="Game id", ascending=False).head(100)

# get the index (matrix) associated to the selected users ids
users = users_table_assoc[users_table_assoc.isin(users_count.index)].index.to_numpy()
users

array([   1,    2,    3,   38,   49,   56,   57,   58,   59,   60,   63,
         64,   66,  117,  119,  121,  140,  144,  182,  185,  186,  224,
        227,  229,  240,  258,  268,  314,  316,  317,  318,  319,  320,
        321,  325,  372,  394,  399,  405,  429,  430,  469,  495,  529,
        584,  585,  586,  589,  647,  675,  742,  752,  753,  783,  784,
        785,  786,  787,  788,  828,  830,  831,  833,  834,  836,  837,
        838,  839,  848,  865,  868,  869,  870,  871,  873,  922,  941,
        942, 1026, 1027, 1053, 1055, 1056, 1060, 1079, 1113, 1114, 1117,
       1118, 1125, 1138, 1208, 1209, 1278, 1307, 1309, 1313, 1381, 1588,
       1589])

In [38]:
# vect_rmse = np.vectorize(calc_RMSE_cos, excluded=['matrix_ratings', 'mask_ratings', 'similarity_matrix'])
# result = vect_rmse(users, matrix_ratings=matrix_ratings, mask_ratings=mask_ratings, similarity_matrix=similarity_matrix)

In [40]:
calc_RMSE_cos(1, matrix_ratings=matrix_ratings, mask_ratings=mask_ratings, similarity_matrix=similarity_matrix)

NotImplementedError: We have not yet implemented 1D sparse slices; please index using explicit indices, e.g. `x[:, [0]]`